# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [12]:
# your code here
orders = pd.read_csv("Orders.csv")


"""
identify two groups of customers from the data set. The first group is VIP Customers
whose aggregated expenses at your global chain stores are above the 95th percentile (aka. 0.95 quantile).
The second group is Preferred Customers whose aggregated expenses are between the 75th and 95th percentile.
"""

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [50]:
# your code here
# First calculate total expenditure of each customers on dataset:
agg_customer_exp = orders.groupby(["CustomerID"]).agg({"amount_spent": "sum"}, reset_index = True)

# calculate 95th percentile (0.95 quantile) and 75th percentile (0.75 quantile) of the amount spend 

q95 = agg_customer_exp.quantile(0.95)
q75 = agg_customer_exp.quantile(0.75)

# Now select customer as "VIP" or as preferred using boolean mask:
# identifying vip customer and adding column in agg_customer_exp dataframe as vip_customer, where the value will boolean True or False
vip_customer = (agg_customer_exp > q95)
vip_customer = vip_customer.rename(columns={"amount_spent": 'is_VIP'})

preferred_customer = ((agg_customer_exp >= q75 ) & (agg_customer_exp <= q95))
preferred_customer = preferred_customer.rename(columns={"amount_spent": "is_Preferred"})

# number count of vip_customer
print(vip_customer['is_VIP'].value_counts()[True])

# number count of preferred customer:
print(preferred_customer["is_Preferred"].value_counts()[True])

217
868


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [51]:
# your code here
# merge the orders and vip_customer dataframe and create vip_orders
vip_orders = pd.merge(left = orders,
                      right = vip_customer, 
                      left_on= "CustomerID",
                      right_on = "CustomerID", 
                      how = "inner")

# use aggreagation to count countries 

vip_country = vip_orders[vip_orders["is_VIP"] == True].groupby(["Country"]).agg(num_vip_customers=("CustomerID", "nunique"))


vip_country.sort_values(by = ["num_vip_customers"], ascending = False)

,num_vip_customers
Country,
United Kingdom,177
Germany,10
France,9
Switzerland,3
Spain,2
Portugal,2
Australia,2
EIRE,2
Japan,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [59]:
# your code here
vip_preferred_orders = pd.merge(left = vip_orders,
                      right = preferred_customer, 
                      left_on= "CustomerID",
                      right_on = "CustomerID", 
                      how = "inner")
vip_condition = vip_preferred_orders["is_VIP"] == True
preferred_condition = vip_preferred_orders["is_Preferred"] == True
vip__preferred_country = vip_preferred_orders[vip_condition | preferred_condition].groupby(["Country"]).agg(num_vip_preferred =("CustomerID", "nunique"))
#vip_preferred_country = (vip_preferred_orders[(vip_preferred_orders["is_VIP"] == True)])  or (vip_preferred_orders[(vip_preferred_orders["is_Preferred"] == True)]).groupby(["Country"]).agg(num_vip_customers=("CustomerID", "nunique"))
vip__preferred_country.sort_values(by = ["num_vip_preferred"], ascending = False)                                                                                
#vip_country = vip_orders[vip_orders["is_VIP"] == True].groupby(["Country"]).agg(num_vip_customers=("CustomerID", "nunique"))

,num_vip_preferred
Country,
United Kingdom,932
Germany,39
France,29
Belgium,12
Switzerland,9
Spain,9
Portugal,7
Norway,7
Italy,5
